# Analyzing replicability of connectivity-based multivariate BWAS on the Human Connectome Project dataset

## Imports

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.model_selection import KFold, train_test_split, cross_val_predict, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from joblib import Parallel, delayed
from mlxtend.evaluate import permutation_test
sns.set(rc={"figure.figsize":(4, 2)})
sns.set_style("whitegrid")

## Load HCP data

We load functional network matrices (netmats) from the HCP1200-release, as published on connectomeDB: https://db.humanconnectome.org/
Due to licensoing issues, data is not supplied with the repository, but can be downloaded from the ConnectomeDB.
See [hcp_data/readme.md](hcp_data/readme.md) for more details.

In [3]:
# HCP data can be obtained from the connectomeDB
# data is not part of this repository
subjectIDs = pd.read_csv('hcp_data/subjectIDs.txt', header=None)

netmats_pearson = pd.read_csv('hcp_data/netmats1_correlationZ.txt',
                             sep=' ',
                             header=None)
netmats_pearson['ID'] = subjectIDs[0]
netmats_pearson.set_index('ID', drop=True, inplace=True)


netmats_parcor = pd.read_csv('hcp_data/netmats2_partial-correlation.txt',
                             sep=' ',
                             header=None)
netmats_parcor['ID'] = subjectIDs[0]
netmats_parcor.set_index('ID', drop=True, inplace=True)

behavior = pd.read_csv('hcp_data/hcp1200_behavioral_data.csv')
behavior = behavior.set_index('Subject', drop=True)

# convert age to numeric
age = []
for s in behavior['Age']:
    if s == '36+':
        age.append(36)
    else:
        split = s.split(sep='-')
        age.append(np.mean((float(split[0]), float(split[1]))))

behavior['age'] = age
behavior

,Release,Acquisition,Gender,Age,3T_Full_MR_Compl,T1_Count,T2_Count,3T_RS-fMRI_Count,3T_RS-fMRI_PctCompl,3T_Full_Task_fMRI,...,Odor_Unadj,Odor_AgeAdj,PainIntens_RawScore,PainInterf_Tscore,Taste_Unadj,Taste_AgeAdj,Mars_Log_Score,Mars_Errs,Mars_Final,age
Subject,,,,,,,,,,,,,,,,,,,,,
100004,S900,Q06,M,22-25,False,0,0,0,0.0,False,...,101.12,86.45,2.0,45.9,107.17,105.31,1.80,0.0,1.80,23.5
100206,S900,Q11,M,26-30,True,1,1,4,100.0,True,...,108.79,97.19,1.0,49.7,72.63,72.03,1.84,0.0,1.84,28.0
100307,Q1,Q01,F,26-30,True,1,1,4,100.0,True,...,101.12,86.45,0.0,38.6,71.69,71.76,1.76,0.0,1.76,28.0
100408,Q3,Q03,M,31-35,True,1,1,4,100.0,True,...,108.79,98.04,2.0,52.6,114.01,113.59,1.76,2.0,1.68,33.0
100610,S900,Q08,M,26-30,True,2,1,4,100.0,True,...,122.25,110.45,0.0,38.6,84.84,85.31,1.92,1.0,1.88,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992774,Q2,Q02,M,31-35,True,2,2,4,100.0,True,...,122.25,111.41,4.0,50.1,107.17,103.55,1.76,0.0,1.76,33.0
993675,S900,Q09,F,26-30,True,2,2,4,100.0,True,...,122.25,110.45,0.0,38.6,84.07,84.25,1.80,1.0,1.76,28.0
994273,S500,Q06,M,26-30,True,1,1,4,100.0,True,...,122.25,111.41,7.0,63.8,110.65,109.73,1.80,1.0,1.76,28.0


# Function to prepare target variable


In [11]:
def create_data(target='CogTotalComp_AgeAdj', feature_data=netmats_parcor):
    # it's a good practice to use pandas for merging, messing up subject order can be painful
    features = feature_data.columns
    df = behavior
    df = df.merge(feature_data, left_index=True, right_index=True, how='left')

    df = df.dropna(subset = [target] + features.values.tolist())
    y = df[target].values
    X = df[features].values
    return X, y

# Function implementing a single bootstrap iteration

We define a workhorse function which:
- randomly samples the discovery and the replication datasets,
- creates cross-validated estimates of predictive performance within the discovery sample
- finalizes the model by fitting it to the whole discovery sample (overfits the discovery but not the replication sample)
- use it to predict the replication sample

In [12]:
def bootstrap_workhorse(X, y, sample_size, model, random_state, shuffle_y=False):

    #create discovery and replication samples by random sampling from the whole dataset (without replacement)

    if shuffle_y:
        rng = np.random.default_rng(random_state)
        y = rng.permutation(y)

    X_discovery, X_replication, y_discovery, y_replication = train_test_split(X, y, train_size=sample_size, test_size=sample_size, shuffle=True, random_state=random_state)

    cv = KFold(10)
    # obtain cross-validated predictions in the discovery sample

    predicted_discovery_cv = np.zeros_like(y_discovery)
    cor_per_fold = np.zeros(cv.n_splits)
    i = 0
    for train, test in cv.split(X=X_discovery, y=y_discovery):
        model.fit(X=X_discovery[train], y=y_discovery[train])
        predicted_discovery_cv[test] = model.predict(X=X_discovery[test])
        cor_per_fold[i] = np.corrcoef(y_discovery[test], predicted_discovery_cv[test])[0,1]
        i += 1
    # correlation between the cross-validated predictions and observations in the discovery sample
    # this is the correct, unbiased estimate!
    # calculated as mean test performance across all folds
    r_disc_cv = np.mean(cor_per_fold)
    # finalize model by training it on the full discovery sample (without cross-validation)
    final_model = model.fit(X=X_discovery, y=y_discovery)
    # obtain predictions with the final model on the discovery sample, note that this model actually overfits this sample.
    # we do this only to demonstrate biased estimates
    predicted_discovery_overfit = final_model.predict(X=X_discovery)
    # here we obtain the biased effect size (r) estimates for demonstrational purposes
    r_disc_overfit = np.corrcoef(predicted_discovery_overfit, y_discovery)[0, 1]

    # We use the final model to predict the replication sample
    # This is correct (no overfitting here), the final model did not see this data during training
    predicted_replication = final_model.predict(X=X_replication)
    # we obtain the out-of-sample prediction performance estimates
    r_rep = np.corrcoef(predicted_replication, y_replication)[0, 1]

    # below we calculate permutation-based p-values for all three effect size estimates (in-sample unbiased, in-sample biased, out-of-sample)
    # (one sided tests, testing for positive correlation)
    p_disc_cv = permutation_test(predicted_discovery_cv, y_discovery, method='approximate', num_rounds=1000, func=lambda x, y: np.corrcoef(x, y)[1][0],seed=random_state)
    p_disc_overfit = permutation_test(predicted_discovery_overfit, y_discovery, method='approximate', num_rounds=1000, func=lambda x, y: np.corrcoef(x, y)[1][0],seed=random_state)
    p_rep = permutation_test(predicted_replication, y_replication, method='approximate', num_rounds=1000, func=lambda x, y: np.corrcoef(x, y)[1][0],seed=random_state)
    # return results
    return r_disc_cv, r_disc_overfit, r_rep, p_disc_cv, p_disc_overfit, p_rep

All set, now we start the analysis.

# Replicability with sample sizes n=50, 100, 200, 300 and max
Here we train a few different models on 100 bootstrap samples.

We aggregate the results of our workhorse function in `n_bootstrap`=100 bootstrap cases (run in parallel).

The whole process is repeated for all sample sizes, fetaure_sets and target variables.

## Here we test age and 5 cognitive variables, including 'cognitive ability' (the main target variable in the target paper)
- age: age group of the participants
- CogTotalComp_AgeAdj: total cognitive ability
- PMAT24_A_CR, : Fluid Intelligence (Penn Progressive Matrices)
- CardSort_AgeAdj: Executive Function/Cognitive Flexibility (Dimensional Change Card Sort)
- Flanker_AgeAdj: Executive Function/Inhibition (Flanker Task)
- PicSeq_AgeAdj: Episodic Memory (Picture Sequence Memory)

# Reproducing the PCA+SVR-based model from the target paper
### Like in the target paper:
- Both PCA and SVR are done inside the cross-validation
- PCA reatains the firts k principal components that together explain 50% of the variance
- scikit-learn makes sure that PCA is only fit for the training samples
- both for the test sets (in the cross-validation) and the replication sample PCA is not re-fit, bt features are simply transformed with the already fit PCA

In [6]:
%%time

random_state = 42
n_bootstrap = 100

features = {
    'netmats_parcor': netmats_parcor,
    'netmats_pearson': netmats_pearson
}

models = {
    'PCA_SVR': Pipeline([('pca', PCA(n_components=0.5)),
                         ('svr', SVR())])

}

# We aggregate all results here:
df = pd.DataFrame(columns=['connectivity','model','target','n','r_discovery_cv','r_discovery_overfit','r_replication','p_discovery_cv','p_discovery_overfit','p_replication'])

for feature_set in features:
    for model in models:
        for target_var in ['age', 'CogTotalComp_AgeAdj', 'PMAT24_A_CR', 'Flanker_AgeAdj', 'CardSort_AgeAdj', 'PicSeq_AgeAdj']:
            for sample_size in [50, 100, 200, 300, 'max']:

                print('*****************************************************************')
                print(feature_set, model, target_var, sample_size)

                X, y = create_data(target=target_var, feature_data=features[feature_set])

                if sample_size=='max':
                    sample_size = int(len(y)/2)

                # create random seeds for each bootstrap iteration for reproducibility
                rng = np.random.default_rng(random_state)
                random_sates = rng.integers(np.iinfo(np.int32).max, size=n_bootstrap)

                # run bootstrap iterations in parallel
                r_discovery_cv, r_discovery_overfit, r_replication, p_discovery_cv, p_discovery_overfit, p_replication = zip(
                    *Parallel(n_jobs=-1)(
                    delayed(bootstrap_workhorse)(X, y, sample_size, models[model], seed) for seed in random_sates))

                tmp_data_frame = pd.DataFrame({
                    'connectivity' : feature_set,
                    'model' : model,
                    'target' : target_var,
                    'n' : sample_size,
                    'r_discovery_cv': r_discovery_cv,
                    'r_discovery_overfit': r_discovery_overfit,
                    'r_replication': r_replication,
                    'p_discovery_cv': p_discovery_cv,
                    'p_discovery_overfit': p_discovery_overfit,
                    'p_replication': p_replication
                })
                #sns.scatterplot(x='r_replication', y='r_discovery_cv', data=tmp_data_frame)
                #plt.ylabel('in-sample (r)')
                #plt.xlabel('out-of-sample (r_pred)')
                #plt.show()
                print(tmp_data_frame.r_discovery_cv.mean(), tmp_data_frame.r_replication.mean())

                for alpha in [0.05, 0.01, 0.005, 0.001]:
                    print('Replicability at alpha =', alpha, ':',
                          (tmp_data_frame.loc[tmp_data_frame['p_discovery_cv']<alpha,'p_replication']<alpha).sum() / (tmp_data_frame['p_discovery_cv']<0.05).sum() * 100, '%')

                df = pd.concat((df, tmp_data_frame))
                df.reset_index(drop=True, inplace=True)
                df.to_csv('res/results_PCA_SVR.csv')

df

*****************************************************************
netmats_parcor PCA_SVR age 50
0.18451221232892587 0.18901378266057708
Replicability at alpha = 0.05 : 57.14285714285714 %
Replicability at alpha = 0.01 : 14.285714285714285 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor PCA_SVR age 100
0.24729849550421484 0.27347377929445293
Replicability at alpha = 0.05 : 89.55223880597015 %
Replicability at alpha = 0.01 : 40.298507462686565 %
Replicability at alpha = 0.005 : 31.343283582089555 %
Replicability at alpha = 0.001 : 11.940298507462686 %
*****************************************************************
netmats_parcor PCA_SVR age 200
0.34201730090591836 0.36175079133098437
Replicability at alpha = 0.05 : 100.0 %
Replicability at alpha = 0.01 : 98.9795918367347 %
Replicability at alpha = 0.005 : 95.91836734693877 %
Replicability at alpha = 0.001 : 80.61224489795919

<timed exec>:60: RuntimeWarning: invalid value encountered in long_scalars


0.19005811228506242 0.21262859874092135
Replicability at alpha = 0.05 : 90.9090909090909 %
Replicability at alpha = 0.01 : 45.45454545454545 %
Replicability at alpha = 0.005 : 18.181818181818183 %
Replicability at alpha = 0.001 : 9.090909090909092 %
*****************************************************************
netmats_parcor PCA_SVR CogTotalComp_AgeAdj 200
0.2623738461298855 0.2713589974517626
Replicability at alpha = 0.05 : 100.0 %
Replicability at alpha = 0.01 : 61.29032258064516 %
Replicability at alpha = 0.005 : 53.2258064516129 %
Replicability at alpha = 0.001 : 27.419354838709676 %
*****************************************************************
netmats_parcor PCA_SVR CogTotalComp_AgeAdj 300
0.30318564055636044 0.3101363215395622
Replicability at alpha = 0.05 : 100.0 %
Replicability at alpha = 0.01 : 94.73684210526315 %
Replicability at alpha = 0.005 : 94.73684210526315 %
Replicability at alpha = 0.001 : 76.84210526315789 %
***************************************************

<timed exec>:60: RuntimeWarning: invalid value encountered in long_scalars


0.0386050459023509 0.04446606264674254
Replicability at alpha = 0.05 : 0.0 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor PCA_SVR Flanker_AgeAdj 200
0.04725917071923175 0.05833533752802022
Replicability at alpha = 0.05 : 33.33333333333333 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor PCA_SVR Flanker_AgeAdj 300
0.07312846509978996 0.07671340073350896
Replicability at alpha = 0.05 : 26.666666666666668 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor PCA_SVR Flanker_AgeAdj max
0.11378620577769874 0.12250618953881152
Replicability at alpha = 0

...

<timed exec>:60: RuntimeWarning: invalid value encountered in long_scalars


0.06624026266001 0.05511154998176417
Replicability at alpha = 0.05 : 60.0 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_pearson PCA_SVR CardSort_AgeAdj 200
0.0753583027418951 0.07325950038420452
Replicability at alpha = 0.05 : 45.0 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_pearson PCA_SVR CardSort_AgeAdj 300
0.08376531584402028 0.09085835147403382
Replicability at alpha = 0.05 : 38.46153846153847 %
Replicability at alpha = 0.01 : 3.8461538461538463 %
Replicability at alpha = 0.005 : 3.8461538461538463 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_pearson PCA_SVR CardSort_AgeAdj max
0.10509717705029202 0.10638876826754434
Replic

<timed exec>:60: RuntimeWarning: invalid value encountered in long_scalars


0.047759076983809795 0.06147545482107202
Replicability at alpha = 0.05 : 20.0 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_pearson PCA_SVR PicSeq_AgeAdj 200
0.05474657301765336 0.06749833856170852
Replicability at alpha = 0.05 : 42.857142857142854 %
Replicability at alpha = 0.01 : 14.285714285714285 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_pearson PCA_SVR PicSeq_AgeAdj 300
0.06991029950177208 0.07458636299063129
Replicability at alpha = 0.05 : 41.66666666666667 %
Replicability at alpha = 0.01 : 8.333333333333332 %
Replicability at alpha = 0.005 : 8.333333333333332 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_pearson PCA_SVR PicSeq_AgeAdj max
0.09227559512484998 

,connectivity,model,target,n,r_discovery_cv,r_discovery_overfit,r_replication,p_discovery_cv,p_discovery_overfit,p_replication
0,netmats_parcor,PCA_SVR,age,50,NaN,0.774355,0.204078,0.997003,0.000999,0.091908
1,netmats_parcor,PCA_SVR,age,50,0.153087,0.913355,0.36097,0.275724,0.000999,0.001998
2,netmats_parcor,PCA_SVR,age,50,NaN,0.841646,0.265158,0.962038,0.000999,0.025974
3,netmats_parcor,PCA_SVR,age,50,0.27227,0.899457,-0.206159,0.374625,0.000999,0.923077
4,netmats_parcor,PCA_SVR,age,50,0.34943,0.892774,0.282873,0.196803,0.000999,0.026973
...,...,...,...,...,...,...,...,...,...,...
5995,netmats_pearson,PCA_SVR,PicSeq_AgeAdj,501,0.170381,0.360809,0.062856,0.004995,0.000999,0.085914
5996,netmats_pearson,PCA_SVR,PicSeq_AgeAdj,501,0.095436,0.352693,0.142834,0.085914,0.000999,0.000999
5997,netmats_pearson,PCA_SVR,PicSeq_AgeAdj,501,0.098129,0.373386,0.163908,0.028971,0.000999,0.000999
5998,netmats_pearson,PCA_SVR,PicSeq_AgeAdj,501,0.12842,0.376163,0.109268,0.065934,0.000999,0.012987


# Now we fit a simple Ridge regression
(no feature selection, no hyperparameter optimization)
This can be expected to perform better on low samples than SVR.

In [7]:
%%time

random_state = 42
n_bootstrap = 100

features = {
    'netmats_parcor': netmats_parcor,
    'netmats_pearson': netmats_pearson
}

models = {
    'ridge': Ridge()
}

# We aggregate all results here:
df = pd.DataFrame(columns=['connectivity','model','target','n','r_discovery_cv','r_discovery_overfit','r_replication','p_discovery_cv','p_discovery_overfit','p_replication'])

for feature_set in features:
    for model in models:
        for target_var in ['age', 'CogTotalComp_AgeAdj', 'PMAT24_A_CR', 'Flanker_AgeAdj', 'CardSort_AgeAdj', 'PicSeq_AgeAdj']:
            for sample_size in [50, 100, 200, 300, 'max']:

                print('*****************************************************************')
                print(feature_set, model, target_var, sample_size)

                X, y = create_data(target=target_var, feature_data=features[feature_set])

                if sample_size=='max':
                    sample_size = int(len(y)/2)

                # create random seeds for each bootstrap iteration for reproducibility
                rng = np.random.default_rng(random_state)
                random_sates = rng.integers(np.iinfo(np.int32).max, size=n_bootstrap)

                # run bootstrap iterations in parallel
                r_discovery_cv, r_discovery_overfit, r_replication, p_discovery_cv, p_discovery_overfit, p_replication = zip(
                    *Parallel(n_jobs=-1)(
                    delayed(bootstrap_workhorse)(X, y, sample_size, models[model], seed) for seed in random_sates))

                tmp_data_frame = pd.DataFrame({
                    'connectivity' : feature_set,
                    'model' : model,
                    'target' : target_var,
                    'n' : sample_size,
                    'r_discovery_cv': r_discovery_cv,
                    'r_discovery_overfit': r_discovery_overfit,
                    'r_replication': r_replication,
                    'p_discovery_cv': p_discovery_cv,
                    'p_discovery_overfit': p_discovery_overfit,
                    'p_replication': p_replication
                })
                #sns.scatterplot(x='r_replication', y='r_discovery_cv', data=tmp_data_frame)
                #plt.ylabel('in-sample (r)')
                #plt.xlabel('out-of-sample (r_pred)')
                #plt.show()
                print(tmp_data_frame.r_discovery_cv.mean(), tmp_data_frame.r_replication.mean())

                for alpha in [0.05, 0.01, 0.005, 0.001]:
                    print('Replicability at alpha =', alpha, ':',
                          (tmp_data_frame.loc[tmp_data_frame['p_discovery_cv']<alpha,'p_replication']<alpha).sum() / (tmp_data_frame['p_discovery_cv']<0.05).sum() * 100, '%')

                df = pd.concat((df, tmp_data_frame))
                df.reset_index(drop=True, inplace=True)
                df.to_csv('res/results_Ridge.csv')

df


*****************************************************************
netmats_parcor ridge age 50
0.24233370132686197 0.2609198136325508
Replicability at alpha = 0.05 : 58.536585365853654 %
Replicability at alpha = 0.01 : 14.634146341463413 %
Replicability at alpha = 0.005 : 12.195121951219512 %
Replicability at alpha = 0.001 : 7.317073170731707 %
*****************************************************************
netmats_parcor ridge age 100
0.3323209164524509 0.34287580012326385
Replicability at alpha = 0.05 : 97.75280898876404 %
Replicability at alpha = 0.01 : 71.91011235955057 %
Replicability at alpha = 0.005 : 58.42696629213483 %
Replicability at alpha = 0.001 : 38.20224719101123 %
*****************************************************************
netmats_parcor ridge age 200
0.39528891792691084 0.4213171713707975
Replicability at alpha = 0.05 : 100.0 %
Replicability at alpha = 0.01 : 100.0 %
Replicability at alpha = 0.005 : 99.0 %
Replicability at alpha = 0.001 : 97.0 %
****************

,connectivity,model,target,n,r_discovery_cv,r_discovery_overfit,r_replication,p_discovery_cv,p_discovery_overfit,p_replication
0,netmats_parcor,ridge,age,50,NaN,1.0,0.404661,0.629371,0.000999,0.002997
1,netmats_parcor,ridge,age,50,0.082894,1.0,0.281041,0.210789,0.000999,0.026973
2,netmats_parcor,ridge,age,50,NaN,1.0,0.107181,0.227772,0.000999,0.216783
3,netmats_parcor,ridge,age,50,0.356733,1.0,-0.203981,0.047952,0.000999,0.909091
4,netmats_parcor,ridge,age,50,0.455145,1.0,0.385665,0.001998,0.000999,0.004995
...,...,...,...,...,...,...,...,...,...,...
5995,netmats_pearson,ridge,PicSeq_AgeAdj,501,0.197442,1.0,0.217841,0.000999,0.000999,0.000999
5996,netmats_pearson,ridge,PicSeq_AgeAdj,501,0.243984,1.0,0.20198,0.000999,0.000999,0.000999
5997,netmats_pearson,ridge,PicSeq_AgeAdj,501,0.263024,1.0,0.203847,0.000999,0.000999,0.000999
5998,netmats_pearson,ridge,PicSeq_AgeAdj,501,0.167095,1.0,0.195714,0.001998,0.000999,0.000999


# Null scenario with random target
To evaluate false positives with biased estimates

In [14]:
%%time

random_state = 42
n_bootstrap = 100

features = {
    'netmats_parcor': netmats_parcor,
    'netmats_pearson': netmats_pearson
}

models = {
    'PCA_SVR': Pipeline([('pca', PCA(n_components=0.5)),
                         ('svr', SVR())])

}

# We aggregate all results here:
df = pd.DataFrame(columns=['connectivity','model','target','n','r_discovery_cv','r_discovery_overfit','r_replication','p_discovery_cv','p_discovery_overfit','p_replication'])

for feature_set in features:
    for model in models:
        for target_var in ['age', 'CogTotalComp_AgeAdj', 'PMAT24_A_CR', 'Flanker_AgeAdj', 'CardSort_AgeAdj', 'PicSeq_AgeAdj']:
            for sample_size in [50, 100, 200, 300, 'max']:

                print('*****************************************************************')
                print(feature_set, model, target_var, sample_size)

                X, y = create_data(target=target_var, feature_data=features[feature_set]) # gives a random y when target is None

                if sample_size=='max':
                    sample_size = int(len(y)/2)

                # create random seeds for each bootstrap iteration for reproducibility
                rng = np.random.default_rng(random_state)
                random_sates = rng.integers(np.iinfo(np.int32).max, size=n_bootstrap)

                # run bootstrap iterations in parallel, with shuffle_y=True
                r_discovery_cv, r_discovery_overfit, r_replication, p_discovery_cv, p_discovery_overfit, p_replication = zip(
                    *Parallel(n_jobs=-1)(
                    delayed(bootstrap_workhorse)(X, y, sample_size, models[model], seed, shuffle_y=True) for seed in random_sates))

                tmp_data_frame = pd.DataFrame({
                    'connectivity' : feature_set,
                    'model' : model,
                    'target' : target_var,
                    'n' : sample_size,
                    'r_discovery_cv': r_discovery_cv,
                    'r_discovery_overfit': r_discovery_overfit,
                    'r_replication': r_replication,
                    'p_discovery_cv': p_discovery_cv,
                    'p_discovery_overfit': p_discovery_overfit,
                    'p_replication': p_replication
                })

                #sns.scatterplot(x='r_replication', y='r_discovery_cv', data=tmp_data_frame)
                #plt.ylabel('in-sample (r)')
                #plt.xlabel('out-of-sample (r_pred)')
                #plt.show()
                print(tmp_data_frame.r_discovery_cv.mean(), tmp_data_frame.r_replication.mean())

                for alpha in [0.05, 0.01, 0.005, 0.001]:
                    print('Replicability at alpha =', alpha, ':',
                          (tmp_data_frame.loc[tmp_data_frame['p_discovery_cv']<alpha,'p_replication']<alpha).sum() / (tmp_data_frame['p_discovery_cv']<0.05).sum() * 100, '%')

                df = pd.concat((df, tmp_data_frame))
                df.reset_index(drop=True, inplace=True)
                df.to_csv('res/results_null_PCA_SVR.csv')

df

*****************************************************************
netmats_parcor PCA_SVR age 50
0.015718268671406327 -0.009632305254739223
Replicability at alpha = 0.05 : 0.0 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor PCA_SVR age 100
0.02118342357629429 0.0008314530617750957
Replicability at alpha = 0.05 : 0.0 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor PCA_SVR age 200
0.0020095024919472414 -0.0020659580620204566
Replicability at alpha = 0.05 : 12.5 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor PCA_SVR age 300
0.010896406643841883

<timed exec>:61: RuntimeWarning: invalid value encountered in long_scalars


0.005333043157679098 -0.012089870875584273
Replicability at alpha = 0.05 : 0.0 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor PCA_SVR CogTotalComp_AgeAdj 200
-0.0017982202571123606 -0.012032126906222016
Replicability at alpha = 0.05 : 0.0 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor PCA_SVR CogTotalComp_AgeAdj 300
0.008405711567388275 -0.009813451575864182
Replicability at alpha = 0.05 : 0.0 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor PCA_SVR CogTotalComp_AgeAdj max
-0.007774668229775042 -0.002075140122022788
Replicability at alpha =

<timed exec>:61: RuntimeWarning: invalid value encountered in long_scalars


0.007738730008135625 -0.020071926806312933
Replicability at alpha = 0.05 : 0.0 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor PCA_SVR PMAT24_A_CR 100
0.013192802109739422 -0.013371222236656783
Replicability at alpha = 0.05 : 0.0 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor PCA_SVR PMAT24_A_CR 200
0.0010659245161563945 0.00011201143802545263
Replicability at alpha = 0.05 : 0.0 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor PCA_SVR PMAT24_A_CR 300
0.01700325765194698 -0.00932689468166226
Replicability at alpha = 0.05 : 0.0 %
Replicability

...

<timed exec>:61: RuntimeWarning: invalid value encountered in long_scalars


-0.0066700364064274445 -0.0012550854807044523
Replicability at alpha = 0.05 : 0.0 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_pearson PCA_SVR CardSort_AgeAdj 200
-0.007011765164321644 -0.001359026986178459
Replicability at alpha = 0.05 : 33.33333333333333 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_pearson PCA_SVR CardSort_AgeAdj 300
0.014291142572627707 -0.00043760039646580204
Replicability at alpha = 0.05 : 0.0 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_pearson PCA_SVR CardSort_AgeAdj max
7.973534426850258e-05 0.0009197645946550262
Replicability a

<timed exec>:61: RuntimeWarning: invalid value encountered in long_scalars


-0.006270139331591028 0.001866435500016127
Replicability at alpha = 0.05 : 0.0 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_pearson PCA_SVR PicSeq_AgeAdj 200
-0.005344849015655386 0.0013732274285362875
Replicability at alpha = 0.05 : 0.0 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_pearson PCA_SVR PicSeq_AgeAdj 300
0.010393117348453423 0.0005262481184145984
Replicability at alpha = 0.05 : 0.0 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_pearson PCA_SVR PicSeq_AgeAdj max
-0.00039147984484469634 0.0016130406528689545
Replicability at alpha = 0.05 : 0.0 %


,connectivity,model,target,n,r_discovery_cv,r_discovery_overfit,r_replication,p_discovery_cv,p_discovery_overfit,p_replication
0,netmats_parcor,PCA_SVR,age,50,-0.010709,0.919133,0.140633,0.577423,0.000999,0.157842
1,netmats_parcor,PCA_SVR,age,50,NaN,0.791907,-0.074325,0.911089,0.000999,0.71029
2,netmats_parcor,PCA_SVR,age,50,-0.354673,0.872867,-0.253604,0.999001,0.000999,0.965035
3,netmats_parcor,PCA_SVR,age,50,-0.076261,0.886898,-0.079072,0.99001,0.000999,0.695305
4,netmats_parcor,PCA_SVR,age,50,-0.139239,0.887541,-0.021112,0.947053,0.000999,0.585415
...,...,...,...,...,...,...,...,...,...,...
5995,netmats_pearson,PCA_SVR,PicSeq_AgeAdj,501,0.017149,0.334397,-0.004167,0.796204,0.000999,0.542458
5996,netmats_pearson,PCA_SVR,PicSeq_AgeAdj,501,-0.090964,0.325431,-0.060089,0.99001,0.000999,0.899101
5997,netmats_pearson,PCA_SVR,PicSeq_AgeAdj,501,0.073699,0.307215,-0.083865,0.23976,0.000999,0.973027
5998,netmats_pearson,PCA_SVR,PicSeq_AgeAdj,501,0.039265,0.311807,-0.081005,0.305694,0.000999,0.962038


In [13]:
%%time

random_state = 42
n_bootstrap = 100

features = {
    'netmats_parcor': netmats_parcor,
    'netmats_pearson': netmats_pearson
}

models = {
    'Ridge': Ridge()

}

# We aggregate all results here:
df = pd.DataFrame(columns=['connectivity','model','target','n','r_discovery_cv','r_discovery_overfit','r_replication','p_discovery_cv','p_discovery_overfit','p_replication'])

for feature_set in features:
    for model in models:
        for target_var in ['age', 'CogTotalComp_AgeAdj', 'PMAT24_A_CR', 'Flanker_AgeAdj', 'CardSort_AgeAdj', 'PicSeq_AgeAdj']:
            for sample_size in [50, 100, 200, 300, 'max']:

                print('*****************************************************************')
                print(feature_set, model, target_var, sample_size)

                X, y = create_data(target=target_var, feature_data=features[feature_set]) # gives a random y when target is None

                if sample_size=='max':
                    sample_size = int(len(y)/2)

                # create random seeds for each bootstrap iteration for reproducibility
                rng = np.random.default_rng(random_state)
                random_sates = rng.integers(np.iinfo(np.int32).max, size=n_bootstrap)

                # run bootstrap iterations in parallel, with shuffle_y=True
                r_discovery_cv, r_discovery_overfit, r_replication, p_discovery_cv, p_discovery_overfit, p_replication = zip(
                    *Parallel(n_jobs=-1)(
                    delayed(bootstrap_workhorse)(X, y, sample_size, models[model], seed, shuffle_y=True) for seed in random_sates))

                tmp_data_frame = pd.DataFrame({
                    'connectivity' : feature_set,
                    'model' : model,
                    'target' : target_var,
                    'n' : sample_size,
                    'r_discovery_cv': r_discovery_cv,
                    'r_discovery_overfit': r_discovery_overfit,
                    'r_replication': r_replication,
                    'p_discovery_cv': p_discovery_cv,
                    'p_discovery_overfit': p_discovery_overfit,
                    'p_replication': p_replication
                })

                #sns.scatterplot(x='r_replication', y='r_discovery_cv', data=tmp_data_frame)
                #plt.ylabel('in-sample (r)')
                #plt.xlabel('out-of-sample (r_pred)')
                #plt.show()
                print(tmp_data_frame.r_discovery_cv.mean(), tmp_data_frame.r_replication.mean())

                for alpha in [0.05, 0.01, 0.005, 0.001]:
                    print('Replicability at alpha =', alpha, ':',
                          (tmp_data_frame.loc[tmp_data_frame['p_discovery_cv']<alpha,'p_replication']<alpha).sum() / (tmp_data_frame['p_discovery_cv']<0.05).sum() * 100, '%')

                df = pd.concat((df, tmp_data_frame))
                df.reset_index(drop=True, inplace=True)
                df.to_csv('res/results_null_Ridge.csv')

df

*****************************************************************
netmats_parcor Ridge age 50
-0.014348756240858624 -0.019865509971863777
Replicability at alpha = 0.05 : 0.0 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor Ridge age 100
0.011020054799004317 0.00818911468614934
Replicability at alpha = 0.05 : 0.0 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor Ridge age 200
0.014167298702132609 -0.0043645513332393575
Replicability at alpha = 0.05 : 11.11111111111111 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor Ridge age 300
-0.0021402405014

,connectivity,model,target,n,r_discovery_cv,r_discovery_overfit,r_replication,p_discovery_cv,p_discovery_overfit,p_replication
0,netmats_parcor,Ridge,age,50,-0.051893,1.0,0.033311,0.752248,0.000999,0.400599
1,netmats_parcor,Ridge,age,50,NaN,1.0,-0.09151,0.916084,0.000999,0.755245
2,netmats_parcor,Ridge,age,50,0.00365,1.0,0.171662,0.682318,0.000999,0.121878
3,netmats_parcor,Ridge,age,50,-0.417308,1.0,0.002095,0.999001,0.000999,0.481518
4,netmats_parcor,Ridge,age,50,-0.063412,1.0,-0.016329,0.877123,0.000999,0.574426
...,...,...,...,...,...,...,...,...,...,...
5995,netmats_pearson,Ridge,PicSeq_AgeAdj,501,0.028744,1.0,-0.008928,0.328671,0.000999,0.594406
5996,netmats_pearson,Ridge,PicSeq_AgeAdj,501,-0.02452,1.0,0.004109,0.657343,0.000999,0.462537
5997,netmats_pearson,Ridge,PicSeq_AgeAdj,501,-0.116399,1.0,0.009268,0.991009,0.000999,0.435564
5998,netmats_pearson,Ridge,PicSeq_AgeAdj,501,0.074094,1.0,-0.033243,0.040959,0.000999,0.751249


*See the notebook called 'plot_results.ipynb' for the results.*